In [14]:
#export
from zero.imports import *

### basic blocks, or fundamental components of neural networks

In [15]:
#export
def conv_unit(ni:int, no:int, ks:int=3, stride:int=1, groups:int=1, zero_bn:bool=False, seq:tuple=(1,2,3)):
    """
    The basic unit of convolutional neural networks,
    
    Parameters:
    -----------
    ni : number of input channels for conv operator
    no : number of output channels for conv operator
    ks : kernel size of conv operator
    stride : stride size of conv operator
    groups : number of groups of conv operator
    zero_bn : does initialize zero value for weight of batch norm
    seq : sequence of operators, operators are represented by interges:
          0  :  None
          1  :  nn.Conv2d
          2  :  nn.ReLU
          3  :  nn.BatchNorm2d
          ...
    
    Return:
    -------
    a nn.Sequential object
    
    """
    unit = []
    has_conv = False # if has conv operator
    for e in seq:
        if e == 0:  # None operator
            continue
        elif e == 1:  # conv operator
            has_conv = True
            unit += [nn.Conv2d(ni, no, ks, stride=stride, padding=ks//2, groups=groups, bias=False)]
        elif e == 2:  # relu operator
            unit += [nn.ReLU(inplace=True)]
        elif e == 3:  # bn operator
            if has_conv: # if has conv operator
                bn = nn.BatchNorm2d(no)  # bn operator's `ni` equal to 'no' of conv op
                nn.init.constant_(bn.weight, 0. if zero_bn else 1.) # zero bn only after conv
                unit += [bn]
            else:  # if has not conv operator
                unit += [nn.BatchNorm2d(ni)] # bn operator's `ni` equal to 'ni' of conv op
    return nn.Sequential(*unit)


In [16]:
#export
# several customized conv units
relu_conv_bn = partial(conv_unit, seq = (2,1,3))  # Relu-->Conv-->BN
conv_bn_relu = partial(conv_unit, seq = (1,3,2))  # Conv-->BN-->Relu
bn_relu_conv = partial(conv_unit, seq = (3,2,1))  # BN-->Relu-->Conv
relu_conv = partial(conv_unit, seq = (2,1,0))  # Relu-->Conv
conv_bn = partial(conv_unit, seq = (1,3,0))  # Conv-->BN

In [17]:
#export
def resnet_basicblock(ni, no, nh, stride:int=1):
    """
    Basic Unit in Residual Networks, ni == no == nh
    
    Reference:
    ----------
    Deep Residual Learning for Image Recognition:
    https://arxiv.org/abs/1512.03385
    """
    return nn.Sequential(*[*relu_conv_bn(ni, nh, stride=stride).children()], 
                         *[*relu_conv_bn(nh, no).children()])

def resnet_bottleneck(ni, no, nh, stride:int=1, groups:int=1, zero_bn=True):
    """
    Bottleneck Unit in Residual Networks, ni == no > nh
    
    Reference:
    ----------
    Deep Residual Learning for Image Recognition:
    https://arxiv.org/abs/1512.03385
    """
    return nn.Sequential(*[*relu_conv_bn(ni, nh, ks=1).children()],
                         *[*relu_conv_bn(nh, nh, stride=stride, groups=groups).children()],
                         *[*relu_conv_bn(nh, no, ks=1, zero_bn=zero_bn).children()])

In [18]:
#export
# residential block
def preresnet_basicblock(ni, no, nh, stride:int=1):
    """
    Basic Unit in Pre-action Residual Networks, ni == no == nh
    
    Reference:
    ----------
    Identity Mappings in Deep Residual Networks:
    https://arxiv.org/abs/1603.05027
    """
    return nn.Sequential(*[*bn_relu_conv(ni, nh, stride=stride).children()], 
                         *[*bn_relu_conv(nh, no).children()])

def preresnet_bottleneck(ni, no, nh, stride:int=1, groups:int=1, zero_bn=True):
    return nn.Sequential(*[*bn_relu_conv(ni, nh, ks=1).children()],
                         *[*bn_relu_conv(nh, nh, stride=stride, groups=groups).children()],
                         *[*bn_relu_conv(nh, no, ks=1, zero_bn=zero_bn).children()])
    

In [19]:
#export
def xception(ni:int, no:int, nh:int, ks:int=3, stride:int=1, zero_bn:bool=False):
    """
    Basic unit in xception networks.
    
    Reference:
    ----------
    Xception: Deep Learning with Depthwise Separable Convolutions:
    https://arxiv.org/abs/1610.02357
    """
    return nn.Sequential(*[*relu_conv(ni, nh, ks=ks, stride=stride, groups=ni).children()],
                        *[*conv_bn(nh, no, ks=1, zero_bn=zero_bn).children()]
                        )

In [20]:
#test xception
x = torch.randn(2, 10, 32, 32)
m = xception(10, 10, 10, zero_bn=True)
m, m[3].weight

(Sequential(
   (0): ReLU(inplace=True)
   (1): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
   (2): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1), bias=False)
   (3): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 ), Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True))

In [21]:
#export
class Classifier(nn.Module):
    """
    Usually work as the final operator for image processing (classification, object detection, etc.)
    Including:
    an average pooling op, which downsampling image resolution to 1x1
    a linear op, which perform classification.
    """
    def __init__(self, ni, no):
        super(Classifier, self).__init__()
        self.adaptivepool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(ni, no)
        
    def forward(self, x):
        out = self.adaptivepool(x)  # out tensor (N, ni, 1, 1)
        out = out.view(out.size(0), -1)  # out tensor (N, ni)
        out = self.fc(out)  # out tensor (N, no)
        return out

In [22]:
#export
class IdentityMapping(nn.Module):
    """
    Identity Mapping between input and output, four cases:
    1.  stride == 1 and ni == no
        input == output
    2.  stride == 1 and ni != no
        1x1 conv and bn
    3.  stride == 2 and ni == no
        maxpool or avgpool
    4.  stride == 2 and ni != no
        (maxpool or avgpool) and 1x1 conv and bn
        
    """
    def __init__(self, ni:int, no:int, stride:int=1, pooling_type:str='max'):
        super(IdentityMapping, self).__init__()
        assert stride == 1 or stride == 2
        assert pooling_type == 'max' or pooling_type == 'avg'
        unit = []
        if stride == 2:
            if pooling_type == 'max':
                downsample = nn.MaxPool2d(kernel_size=3, stride=stride, padding=1)
            elif pooling_type == 'avg':
                downsample = nn.AvgPool2d(kernel_size=3, stride=stride, padding=1)
            unit.append(downsample)
        if ni != no:
            unit += conv_bn(ni, no, ks=1, zero_bn=False).children()
        self.unit = nn.Sequential(*unit)
    def forward(self, x):
        out = self.unit(x)
        return out

In [23]:
#test IdentityMapping
x = torch.randn(2, 10, 32, 32)
m = IdentityMapping(10, 10, stride=1, pooling_type='max')
m(x).shape

torch.Size([2, 10, 32, 32])

### Help functions for neural networks

In [24]:
#export
# copy from https://github.com/fastai/fastai/blob/master/fastai/vision/models/xresnet.py
def init_cnn(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_cnn(l)


In [25]:
#export
def num_params(net:nn.Module):
    "Number of parameters of a neural network"
    num_params = 0
    for name, param in net.named_parameters():
        num = torch.prod(torch.tensor(param.size()))
        num_params += num
        # print(name, param.size(), num)
    return num_params

### Export

In [26]:
!python notebook2script.py core.ipynb

Converted core.ipynb to zero/core.py
